In [ ]:
import os
import os.path
import pathlib
import glob
from pathlib import Path
import glob
import psutil
import math

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf
import tensorflow_io as tfio

from tensorflow.keras import layers
from tensorflow.keras import models
from IPython import display

import time

# Generate a simple seed with added entropy from the current time
seed = int(time.time() * 1000) % (2**32)

tf.random.set_seed(seed)
np.random.seed(seed)

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [ ]:
import random
data_dir = 'Z:/AutoMusic/output_new_1/'
cache_dir = 'Y:/AutoMusic/cache_4/'
high_up_files = [os.path.join(data_dir+"high_up_1/", f) for f in os.listdir(data_dir+"high_up_1/") if os.path.isfile(os.path.join(data_dir+"high_up_1/", f))]
up_test_file = random.sample(high_up_files, 1)
test_file = up_test_file[0]
# test_file = data_dir+"/high_up/high_up.01_adri_block_and_paul_parsons_-_fall_in_lov_020382_027349.wav"
pioneer_files = 'd:/pioneer/usbanlz'
pioneer_prefix = 'D:'
temp_wav_file = 'Z:/AutoMusic/temp/temp.wav'
mytotalfiles = 934 # I know this from previous runs.
EPOCHS = 200 # using early stopping, but just in case

In [ ]:
# verify and clean the training files
# in case the last run died or there was a mixup of the wrong files in label directories 

import os
import shutil

def validate_and_manage_files(base_path, size_threshold, dry_run=True):
    """
    Validates and manages files in immediate subdirectories:
    - Moves files smaller than `size_threshold` bytes to the directory above `base_path` (if `dry_run` is False).
    - Moves files to a matching directory if possible (if `dry_run` is False).
    - Moves files to the directory above `base_path` if no matching directory exists (if `dry_run` is False).
    - Outputs the file size for files below threshold during both dry run and execution.
    - Prints a message if a subdirectory is clean (no changes required).

    :param base_path: Path to the root directory to clean
    :param size_threshold: Minimum file size (in bytes) for a file to be kept in the correct location
    :param dry_run: If True, simulate actions without actually moving files
    """
    # Determine the directory above the root folder
    parent_dir = os.path.abspath(os.path.join(base_path, os.pardir))
    
    for sub_dir in os.listdir(base_path):
        sub_dir_path = os.path.join(base_path, sub_dir)

        # Ensure it's a directory
        if os.path.isdir(sub_dir_path):
            changes_made = False  # Track whether any changes are made in this subdirectory

            for file_name in os.listdir(sub_dir_path):
                file_path = os.path.join(sub_dir_path, file_name)

                # Ensure it's a file
                if os.path.isfile(file_path):
                    # Get file size
                    file_size = os.path.getsize(file_path)

                    # Check if file size is below threshold
                    if file_size < size_threshold:
                        # Simulate or perform moving small file to the parent directory
                        new_file_path = os.path.join(parent_dir, file_name)
                        if dry_run:
                            print(f"[DRY RUN] Would move '{file_name}' ({file_size} bytes, below {size_threshold} bytes) to '{parent_dir}'")
                        else:
                            shutil.move(file_path, new_file_path)
                            print(f"Moved '{file_name}' ({file_size} bytes, below {size_threshold} bytes) to '{parent_dir}'")
                        changes_made = True
                        continue

                    # Check if the file name starts with the parent directory name
                    if not file_name.startswith(sub_dir):
                        proper_dir_path = os.path.join(base_path, sub_dir)

                        # Simulate or perform moving the file to the proper directory
                        if os.path.exists(proper_dir_path):
                            new_file_path = os.path.join(proper_dir_path, file_name)
                            if dry_run:
                                print(f"[DRY RUN] Would move '{file_name}' ({file_size} bytes) to '{proper_dir_path}'")
                            else:
                                shutil.move(file_path, new_file_path)
                                print(f"Moved '{file_name}' ({file_size} bytes) to '{proper_dir_path}'")
                            changes_made = True
                        else:
                            # Simulate or perform moving the file to the parent directory
                            new_file_path = os.path.join(parent_dir, file_name)
                            if dry_run:
                                print(f"[DRY RUN] Would move '{file_name}' ({file_size} bytes, no matching directory found) to '{parent_dir}'")
                            else:
                                shutil.move(file_path, new_file_path)
                                print(f"Moved '{file_name}' ({file_size} bytes, no matching directory found) to '{parent_dir}'")
                            changes_made = True

            # If no changes were made, print a "clean directory" message
            if not changes_made:
                print(f"The directory '{sub_dir_path}' is clean (no changes required).")

validate_and_manage_files(data_dir, size_threshold=480*1000, dry_run=False)  # Do actions, no test

In [ ]:
# Find the next history file number

historycounter = 1 # start looking at this number as history1.txt

Path("./Histories").mkdir(parents=True, exist_ok=True)

while os.path.isfile("./Histories/history"+str(historycounter)+".txt"):
    historycounter += 1

history_file = "./Histories/history"+str(historycounter)+".txt"
print("Using run history file: "+history_file)

checkpoint_filepath = 'Z:/AutoMusic/checkpoint/automusic'+str(historycounter)+'.h5'
print("Using model save/checkpoint file: "+checkpoint_filepath)

# Roughly figure out if the dataset will fit into memory, or we need to use disk caching

free_mem = psutil.virtual_memory()
free_mem = math.floor(free_mem.available/1024/1024/1024*0.90)

samples_size = sum(f.stat().st_size for f in Path(data_dir).glob('**/*') if f.is_file())
samples_size = math.ceil((samples_size/1024/1024/1024)*1.05)

if samples_size < free_mem:
    print("Sample size of "+str(samples_size)+"GB should fit in free memory of "+str(free_mem)+"GB - using RAM to cache")
    cache_dir = ''
else:
    print("Sample size of "+str(samples_size)+"GB will not fit in free memory of "+str(int(free_mem))+"GB - using cache dir "+cache_dir)

In [ ]:
# List all items in data_dir
commands = np.array(tf.io.gfile.listdir(str(data_dir)))

# Filter to include only directories and those that start with "high_"
commands = np.array([cmd for cmd in commands if tf.io.gfile.isdir(f"{data_dir}/{cmd}") and cmd.startswith("high_")])

print('Directories/Labels:', commands)

In [ ]:
# Use the generated seed in your audio dataset creation
train_ds, val_ds = tf.keras.utils.audio_dataset_from_directory(
    directory=data_dir,
    batch_size=32,
    validation_split=0.2,
    seed=seed,  # Use the entropy-enhanced seed
    output_sequence_length=44100*3,
    subset='both'
)

label_names = np.array(train_ds.class_names)
print()
print("label names:", label_names)


In [ ]:
def squeeze(audio, labels):
  # audio = tf.squeeze(audio, axis=-1)
  audio = audio[:,:,-1]
  return audio, labels

train_ds = train_ds.map(squeeze, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.map(squeeze, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
test_ds = val_ds.shard(num_shards=2, index=0)
val_ds = val_ds.shard(num_shards=2, index=1)

In [ ]:
for example_audio, example_labels in train_ds.take(1):  
  print(example_audio.shape)
  print(example_labels.shape)

In [ ]:
class LogMelSpectrogram(tf.keras.layers.Layer):
    """Compute log-magnitude mel-scaled spectrograms."""

    def __init__(self, sample_rate, fft_size, hop_size, n_mels,
                 f_min=0.0, f_max=None, **kwargs):
        super(LogMelSpectrogram, self).__init__(**kwargs)
        self.sample_rate = sample_rate
        self.fft_size = fft_size
        self.hop_size = hop_size
        self.n_mels = n_mels
        self.f_min = f_min
        self.f_max = f_max if f_max else sample_rate / 2
        self.mel_filterbank = tf.signal.linear_to_mel_weight_matrix(
            num_mel_bins=self.n_mels,
            num_spectrogram_bins=fft_size // 2 + 1,
            sample_rate=self.sample_rate,
            lower_edge_hertz=self.f_min,
            upper_edge_hertz=self.f_max)

    def build(self, input_shape):
        self.non_trainable_weights.append(self.mel_filterbank)
        super(LogMelSpectrogram, self).build(input_shape)

    def call(self, waveforms):
        """Forward pass.

        Parameters
        ----------
        waveforms : tf.Tensor, shape = (None, n_samples)
            A Batch of mono waveforms.

        Returns
        -------
        log_mel_spectrograms : (tf.Tensor), shape = (None, time, freq, ch)
            The corresponding batch of log-mel-spectrograms
        """
        def _tf_log10(x):
            numerator = tf.math.log(x)
            denominator = tf.math.log(tf.constant(10, dtype=numerator.dtype))
            return numerator / denominator

        def power_to_db(magnitude, amin=1e-16, top_db=80.0):
            """
            https://librosa.github.io/librosa/generated/librosa.core.power_to_db.html
            I think this is a function in the TF supports now?
            """
            ref_value = tf.reduce_max(magnitude)
            log_spec = 10.0 * _tf_log10(tf.maximum(amin, magnitude))
            log_spec -= 10.0 * _tf_log10(tf.maximum(amin, ref_value))
            log_spec = tf.maximum(log_spec, tf.reduce_max(log_spec) - top_db)

            return log_spec

        spectrograms = tf.signal.stft(waveforms,
                                      frame_length=self.fft_size,
                                      frame_step=self.hop_size,
                                      pad_end=False)

        magnitude_spectrograms = tf.abs(spectrograms)

        mel_spectrograms = tf.matmul(tf.square(magnitude_spectrograms),
                                     self.mel_filterbank)

        log_mel_spectrograms = power_to_db(mel_spectrograms)

        # add channel dimension
        log_mel_spectrograms = tf.expand_dims(log_mel_spectrograms, 3)

        return log_mel_spectrograms

    def get_config(self):
        config = {
            'fft_size': self.fft_size,
            'hop_size': self.hop_size,
            'n_mels': self.n_mels,
            'sample_rate': self.sample_rate,
            'f_min': self.f_min,
            'f_max': self.f_max,
        }
        config.update(super(LogMelSpectrogram, self).get_config())

        return config

In [ ]:
input_shape = example_audio.shape[1:]
input_length = len(input_shape)
num_labels = len(label_names)

frame_length=1024
frame_step=64 # 128
fft_length=None
sample_rate=44100
duration=3
num_mel_channels=80
freq_min=40 # 200
freq_max=10000 # 8000

def ConvModel(n_classes=num_labels, sample_rate=sample_rate, duration=duration,
              fft_size=frame_length, hop_size=frame_step, n_mels=num_mel_channels):
    
    n_samples = sample_rate * duration
    
    # Accept raw audio data as input
    x = layers.Input(shape=(n_samples,), name='input', dtype='float32')
    y = LogMelSpectrogram(sample_rate, fft_size, hop_size, n_mels, freq_min, freq_max)(x)
    y = layers.Resizing(160,80)(y)
    y = layers.BatchNormalization(axis=2)(y)
    y = layers.GaussianNoise(0.5)(y)
    y = layers.Conv2D(32, 3, activation='relu')(y)
    y = layers.Conv2D(64, 3, activation='relu')(y)
    y = layers.MaxPooling2D()(y)
    y = layers.SpatialDropout2D(0.25)(y)
    y = layers.Conv2D(128, 3, activation='relu')(y)  # Additional Conv2D layer
    y = layers.GlobalAveragePooling2D()(y)  
    y = layers.Flatten()(y)
    y = layers.Dense(128, activation='relu')(y)
    y = layers.GaussianDropout(0.5)(y)
    y = layers.Dense(num_labels)(y)
    
    return tf.keras.Model(inputs=x, outputs=y)

model = ConvModel()

model.summary()

# # Define the learning rate schedule
# initial_learning_rate = 0.01
# lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate,
#     decay_steps=100000,
#     decay_rate=0.96,
#     staircase=True
# )

# # Compile the model with the optimizer using the learning rate schedule
# model.compile(
#     optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
#     loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#     metrics=['accuracy'],
# )
             
# learning_rate = 0.001 # 0.0001

# model.compile(
#     optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
#     loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#     metrics=['accuracy'],
# )

In [ ]:
if cache_dir != '':
    for f in Path(cache_dir).glob('*'):
        try:
            print("Removing cache file "+str(f))
            f.unlink()
        except OSError as e:
            print("Error: %s : %s" % (f, e.strerror))
    
train_ds = train_ds.cache(cache_dir).shuffle(buffer_size=1000, reshuffle_each_iteration=True).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.cache(cache_dir).prefetch(tf.data.AUTOTUNE)
test_ds_precache = test_ds
test_ds = test_ds.cache(cache_dir).prefetch(tf.data.AUTOTUNE)

In [ ]:
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1)

earlystop_callback = tf.keras.callbacks.EarlyStopping(
    verbose=1,
    patience=25,
    min_delta=0.001,
    restore_best_weights=True)


# Define the learning rate schedule
initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=100000,
    decay_rate=0.96,
    staircase=True
)

# Compile the model with the optimizer using the learning rate schedule
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=initial_learning_rate),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

# Train the model
# history = model_optimized.fit(train_ds, training_labels, epochs=50)

# This may be a bad idea:
# we're still retraining on ALL data, but this seems to speed up the epochs needed:
# model.load_weights(checkpoint_filepath) # kick off training with the last run's weights

if (os.path.isfile(f"Z:/AutoMusic/checkpoint/automusic{historycounter}.h5")):
    model.load_weights(f"Z:/AutoMusic/checkpoint/automusic{historycounter}.h5") 
    print(f"Resuming current checkpoint {historycounter} to resume.") 
elif (os.path.isfile(f"Z:/AutoMusic/checkpoint/automusic{historycounter-1}.h5")):
    model.load_weights(f"Z:/AutoMusic/checkpoint/automusic{historycounter-1}.h5")
    print(f"Loading previous checkpoint {historycounter-1} weights.")
    
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[earlystop_callback,checkpoint_callback]
)

model.save(f"Z:/AutoMusic/checkpoint/automusic/automusic_model_{historycounter}.h5")

# shouldn't be needed with restore_best_weights=True
# model.load_weights(checkpoint_filepath) # load the best saved weights... even if they aren't from this run, maybe?

In [ ]:
try:
    # Check if 'history' is defined
    if history:
        metrics = history.history
        plt.figure(figsize=(16,6))

        plt.subplot(1,2,1)
        plt.plot(history.epoch, metrics['loss'], metrics['val_loss'])
        plt.legend(['loss', 'val_loss'])
        plt.ylim([0, max(plt.ylim())])
        plt.xlabel('Epoch')
        plt.ylabel('Loss [CrossEntropy]')

        plt.subplot(1,2,2)
        plt.plot(history.epoch, 100*np.array(metrics['accuracy']), 100*np.array(metrics['val_accuracy']))
        plt.legend(['accuracy', 'val_accuracy'])
        plt.ylim([0, 100])
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy [%]')
    else:
        print("Training Skipped. Validation only.")
except NameError:
    print("The 'history' variable is not defined.")


In [ ]:
y_true = tf.concat(list(test_ds_precache.map(lambda s,lab: lab)), axis=0)
y_pred = model.predict(test_ds_precache)
y_pred = y_pred.argmax(axis=1)

confusion_mtx = tf.math.confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_mtx,
            xticklabels=label_names,
            yticklabels=label_names,
            annot=True, fmt='g')
plt.xlabel('Prediction')
plt.ylabel('Label')
plt.title(f'AutoMusic - Electronic Music Phrase Classifier (Run {historycounter})')
plt.savefig(f'Z:/AutoMusic/AutoMusic_HeatMap_{historycounter}.png')
plt.show()

In [ ]:
x = test_file
x = tf.io.read_file(str(x))
x, sample_rate = tf.audio.decode_wav(x, desired_channels=1, desired_samples=44100*3)
x = tf.squeeze(x, axis=-1)
x = x[tf.newaxis,...]
print(x.shape)

prediction = model(x)
plt.bar(label_names, tf.nn.softmax(prediction[0]))
plt.xticks(rotation=90) 
plt.show()


In [ ]:
import os
import random

def delete_random_files(directory, keep_count):
    # List all files in the directory
    all_files = [os.path.join(directory, f) for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
    
    # Select X random files to keep
    keep_files = random.sample(all_files, keep_count)
    
    # Delete the rest of the files
    for file in all_files:
        if file not in keep_files:
            os.remove(file)
            # print(f"Deleted: {file}")
    
    print("Operation completed successfully.")

# number is number of files to KEEP
# delete_random_files("Z:/AutoMusic/output/high_chorus/", 2500)
# delete_random_files("Z:/AutoMusic/output/high_down/", 2500)
# delete_random_files("Z:/AutoMusic/output/high_intro/", 2500)
# delete_random_files("Z:/AutoMusic/output/high_outro/", 2500)
# delete_random_files("Z:/AutoMusic/output/high_up/", 2500)


In [ ]:
import pyrekordbox
from pyrekordbox.anlz import AnlzFile
from pydub import AudioSegment
from pydub.generators import WhiteNoise
from pydub.effects import speedup, normalize
import hashlib
import os
import random
from pathlib import Path
import logging
l = logging.getLogger("pydub.converter")
l.setLevel(logging.CRITICAL)

myfiles = pyrekordbox.anlz.walk_anlz_paths(pioneer_files)

myfilecounter = 0

with open(history_file,'w') as out:

    for myfoundfile in myfiles:

        try: mydat = AnlzFile.parse_file(myfoundfile[1]['DAT'])
        except: continue
        try: myext = AnlzFile.parse_file(myfoundfile[1]['EXT'])
        except: continue

        mymp3 = mydat.get('PPTH')
        mymp3 = pioneer_prefix + mymp3

        if "Ultimate" in str(mymp3):
            # this is very specific to my library where there's some weird files full of random loops
            continue

        # print(mymp3)

        mylabels = {}
        mylabels['high'] = ['unknown', 'high_intro_', 'high_up_', 'high_down', 'unknown', 'high_chorus_', 'high_outro_', 'unknown', 'unknown', 'unknown', 'unknown']
        mylabels['mid']  = ['unknown', 'mid_intro', 'mid_verse_1', 'mid_verse_2', 'mid_verse_3', 'mid_verse_4', 'mid_verse_5', 'mid_verse_5', 'mid_bridge', 'mid_chorus', 'mid_outro']
        mylabels['low']  = ['unknown', 'low_intro', 'low_verse_1', 'low_verse_1', 'low_verse_1', 'low_verse_2', 'low_verse_2', 'low_verse_2', 'low_bridge', 'low_chorus', 'low_outro']

        try: mytimecode = mydat.get('PQTZ')[2]
        except: continue
        try: mystructures = myext.get('PSSI').entries
        except: continue
        try: mymood = myext.get('PSSI').mood
        except: continue

        if (mymood == 1):
            mymood = "high"
        elif (mymood == 2):
            mymood = "mid"
            continue
        elif (mymood == 3):
            mymood = "low"
            continue
        else:
            continue

        mytimecode = mydat.get('PQTZ')[2]

        myfilelastbeat = myext.get('PSSI').end_beat

        mysimplestructure = {}

        myfilecounter += 1

        for x in range(len(mystructures)):

            mylabel = mystructures[x].kind

            if (mymood == "high"):

                if (mylabel == 1 or mylabel == 5 or mylabel == 6):
                    if (mystructures[x].k1 == 1):
                        mylabelmodifier = "1"
                    else:
                        mylabelmodifier = "2"
                elif (mylabel == 2):
                    if (mystructures[x].k2 == 0 and mystructures[x].k3 == 0):
                        mylabelmodifier = "1"
                    elif (mystructures[x].k2 == 0 and mystructures[x].k3 == 1):
                        mylabelmodifier = "2"
                    elif (mystructures[x].k2 == 1 and mystructures[x].k3 == 0):
                        mylabelmodifier = "3"
                    else:
                        continue
                else:
                    mylabelmodifier = ""

            else:
                mylabelmodifier = ""
            
            mylabel = mylabels[mymood][mylabel]+mylabelmodifier

            myfirstbeat = mystructures[x].beat

            if (x+1<len(mystructures)):
                mylastbeat = mystructures[x+1].beat
            else:
                mylastbeat = myfilelastbeat

            if (mylastbeat+1<len(mytimecode)):
                mylastbeat = mylastbeat
            else:
                mylastbeat = len(mytimecode)-1

            mysegmentbars = (mylastbeat-myfirstbeat)//8
            mysegmentleftovers = (mylastbeat-myfirstbeat)%8

            myfirsttimecode = mytimecode[int(myfirstbeat)]*1000
            mylasttimecode = mytimecode[int(mylastbeat)]*1000

            mysimplestructure[int(myfirsttimecode)] = mylabel
            mysimplestructure[int(mylasttimecode-1)] = mylabel

        mysong = AudioSegment.from_file(mymp3,frame_rate=44100)
        mono_audios = mysong.split_to_mono() 
        mysongmono = mono_audios[0]
        # chomp output with a random start time of 0..2999ms into the file, to give some variations.
        myrandomoffset = random.randint(0,2999)
        # mysongmono = mysongmono.normalize()

        mysongmono[myrandomoffset:].export(temp_wav_file,format="wav")

        x = tf.io.read_file(temp_wav_file)
        x, sample_rate = tf.audio.decode_wav(x, desired_channels=1)
        x = tf.squeeze(x, axis=-1)
        waveform = x

        slices = int(waveform.shape[0] / (44100*3))
        samples = tf.split(waveform[: slices * (44100*3)], slices)

        milliseconds = 0
        right = 0
        wrong = 0
        transitions = 0

        mylabelsseen = []
        fileview = "# "

        currentwrongs = 0

        for sample in samples:

            x = sample[tf.newaxis,...]

            prediction = model(x)

            # plt.bar(label_names, tf.nn.softmax(prediction[0]))
            # plt.show()

            # correct label for random offset above, because file may be skewed 1..2999ms ahead
            #
            res = mysimplestructure.get(milliseconds) or mysimplestructure[
                  min(mysimplestructure.keys(), key = lambda key: abs(key-milliseconds-myrandomoffset))]

            res2 = mysimplestructure.get(milliseconds) or mysimplestructure[
                   min(mysimplestructure.keys(), key = lambda key: abs(key-milliseconds-myrandomoffset+3000))]

            myaactualendlabel = str(res2)

            mypredictedlabel = str(label_names[np.argmax(prediction)])
            myactuallabel = str(res)

            myrandom = random.randint(0,9)
            
            if myactuallabel != myaactualendlabel :

                # print(" ~~~ "+mypredictedlabel+" in transition from "+myactuallabel+" to "+myaactualendlabel
                print("~", end="")
                print("~", end="",file=out)
                transitions += 1
                currentwrongs = 0

            if mypredictedlabel == myactuallabel :

                right += 1
                currentwrongs = 0
                mylabeldir = myactuallabel

                if myrandom == 0:
                    print("C", end="")
                    print("C", end="",file=out)
                    myoutputbasefile = myactuallabel+"."+Path(mymp3).stem+"_"+str(int(milliseconds)-myrandomoffset).rjust(6,'0')+"_"+str(int(milliseconds+3000)-myrandomoffset).rjust(6,'0')
                    mytempsong = mysong[milliseconds:milliseconds+3000]
                    mytempsong.export(data_dir+"/"+myactuallabel+"/"+myoutputbasefile+"_CORRECT.wav", format="wav")
                else:
                    print("+", end="")
                    print("+", end="",file=out)
                        
            else:

                wrong += 1
                
                if (myactuallabel not in mylabelsseen or currentwrongs > 0 or myrandom == 0): # 10% chance of random sampling

                    currentwrongs += 1

                    if milliseconds+3001 < len(mysong):

                        mylabelsseen.append(myactuallabel)
                        mylabeldir = myactuallabel

                        myoutputbasefile = myactuallabel+"."+Path(mymp3).stem+"_"+str(int(milliseconds)-myrandomoffset).rjust(6,'0')+"_"+str(int(milliseconds+3000)-myrandomoffset).rjust(6,'0')

#                         myspeedup = mysong[milliseconds:milliseconds+3000].speedup(1.03)
#                         mynoise = WhiteNoise().to_audio_segment(duration=len(myspeedup)).apply_gain(-20)
#                         myspeedup_noise = myspeedup.overlay(mynoise)

                        mytempsong = mysong[milliseconds:milliseconds+3000]
                        mynoise = WhiteNoise().to_audio_segment(duration=len(mytempsong)).apply_gain(-20)
                        mytempsong_noise = mytempsong.overlay(mynoise)

                        mytempsong.export(data_dir+"/"+myactuallabel+"/"+myoutputbasefile+"_FIXES.wav", format="wav")
#                         myspeedup.export(data_dir+"/"+myactuallabel+"/"+myoutputbasefile+"_speed_FIXES_N.wav", format="wav")
#                         myspeedup_noise.export(data_dir+"/"+myactuallabel+"/"+myoutputbasefile+"_speed_noise_FIXES_N.wav", format="wav")
                        mytempsong_noise.export(data_dir+"/"+myactuallabel+"/"+myoutputbasefile+"_noise_FIXES_N.wav", format="wav")

                        if myrandom == 0:
                            print("R", end="")
                            print("R", end="",file=out)
                            currentwrongs = 0 # we only want one random sample, just to spice things up.
                            # this also functions as a "stop writing sequence of samples early" randomness.
                        else:
                            print("W", end="")
                            print("W", end="",file=out)

                        if currentwrongs > 10: # stop over-feeding a LOT of wrong samples to the model next time. Let myrandom take care of this.
                            currentwrongs = 0

                    else:
                        print("!", end="")
                        print("!", end="",file=out)
                else:
                    print("!", end="")
                    print("!", end="",file=out)

            milliseconds += 3000

        print(" ")
        print(" ",file=out)
        print(str(int(right/(right+wrong)*100))+"% correct - "+str(Path(mymp3).name)+" ("+str(myfilecounter)+" of "+str(mytotalfiles)+")")
        print(str(int(right/(right+wrong)*100))+"% correct - "+str(Path(mymp3).name)+" ("+str(myfilecounter)+" of "+str(mytotalfiles)+")",file=out)
        print(" ")
        print(" ",file=out)

In [ ]:
import os.path
from collections import defaultdict

counter = 113
histories = []
histories_score = defaultdict(int)

print("found history:")

while os.path.isfile("./Histories/history"+str(counter)+".txt"):
    print(str(counter),end=' ')
    histories.append(open("./Histories/history"+str(counter)+".txt",'r').readlines())
    counter += 1
print()

best = 0

with open('./Histories/history_all.txt','w') as out:
    
    for z in range(len(histories[0])):
        
        for x in range(len(histories)):
            if histories[x][z][0].isdigit():
                testbest = int(histories[x][z].rstrip().partition("%")[0])
                if testbest > best:
                    best = testbest
                histories_score[x] += testbest
                
        for x in range(len(histories)):
            if histories[x][z].rstrip() == "":
                print("",file=out)
                break
            else:
                if histories[x][z][0].isdigit():
                    testbest = int(histories[x][z].rstrip().partition("%")[0])
                    if best == testbest:
                        print("> ",file=out,end='')
                    else:
                        print("  ",file=out,end='')
                print(histories[x][z].rstrip(),file=out)
                
        best = 0

print(" --------------- ")

start_counter = 113

checkpoint_dir = 'Z:/AutoMusic/checkpoint/'

for x in range(len(histories_score)):
    print("Run ", end='')
    print(start_counter, end='')
    print(" = ", end='')
    print(round(histories_score[x] / 934, 1), end='')
    print("%")
    start_counter += 1
